In [1]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Verify GPU setup
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version:", tf.__version__)

# Define the base path
base_path = "/content/drive/My Drive/preprocessed"

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Iterate through 'fake' and 'real' subfolders to collect file paths and labels
for label, category in enumerate(['fake', 'real']):
    category_path = os.path.join(base_path, category)
    for file_name in os.listdir(category_path):
        file_path = os.path.join(category_path, file_name)
        file_paths.append(file_path)
        labels.append(label)  # Assign label 0 for 'fake' and 1 for 'real'



Num GPUs Available:  1
TensorFlow version: 2.15.0


In [3]:
# Convert lists to numpy arrays
file_paths = np.array(file_paths)
labels = np.array(labels)

# Convert labels to categorical
labels = to_categorical(labels, num_classes=2)

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)


In [4]:

# Function to load and preprocess images
def load_and_preprocess_image(file_path, label, target_size=(224, 224)):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = preprocess_input(image)  # Preprocess the image for ResNet
    return image, label

# Data Augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(factor=0.15),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomContrast(factor=0.1)
])



In [5]:
# Create training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Create validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)



In [6]:
# Load the ResNet50 model pre-trained on ImageNet and exclude the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers with dropout and regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)  # Add dropout layer
predictions = Dense(2, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)



In [8]:

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with the correct parameter for learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with data augmentation
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5, callbacks=[early_stopping])



Epoch 1/5
188/188 [==============================] - 74s 337ms/step - loss: 4.3781 - accuracy: 0.7341 - val_loss: 1.2583 - val_accuracy: 0.8610
Epoch 2/5
188/188 [==============================] - 61s 323ms/step - loss: 1.0106 - accuracy: 0.7861 - val_loss: 0.6823 - val_accuracy: 0.8524
Epoch 3/5
188/188 [==============================] - 61s 325ms/step - loss: 0.6912 - accuracy: 0.7976 - val_loss: 0.5381 - val_accuracy: 0.8584
Epoch 4/5
188/188 [==============================] - 60s 320ms/step - loss: 0.6277 - accuracy: 0.7944 - val_loss: 0.5613 - val_accuracy: 0.8438
Epoch 5/5
188/188 [==============================] - 61s 324ms/step - loss: 0.5970 - accuracy: 0.7900 - val_loss: 0.4840 - val_accuracy: 0.8544


In [9]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model with the correct parameter for learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model with reduced batch size to avoid memory issues
history_fine = model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[early_stopping])

# Save the trained model
# model.save('resnet_model.h5')

Epoch 1/10
188/188 [==============================] - 66s 326ms/step - loss: 0.4836 - accuracy: 0.8352 - val_loss: 0.3827 - val_accuracy: 0.8850
Epoch 2/10
188/188 [==============================] - 61s 325ms/step - loss: 0.4286 - accuracy: 0.8563 - val_loss: 0.3823 - val_accuracy: 0.8803
Epoch 3/10
188/188 [==============================] - 61s 326ms/step - loss: 0.4115 - accuracy: 0.8623 - val_loss: 0.3621 - val_accuracy: 0.8836
Epoch 4/10
188/188 [==============================] - 62s 328ms/step - loss: 0.3794 - accuracy: 0.8713 - val_loss: 0.4072 - val_accuracy: 0.8644
Epoch 5/10
188/188 [==============================] - 61s 322ms/step - loss: 0.3590 - accuracy: 0.8823 - val_loss: 0.3907 - val_accuracy: 0.8710
Epoch 6/10
188/188 [==============================] - 61s 324ms/step - loss: 0.3479 - accuracy: 0.8852 - val_loss: 0.3730 - val_accuracy: 0.8783
Epoch 7/10
188/188 [==============================] - 61s 324ms/step - loss: 0.3247 - accuracy: 0.8929 - val_loss: 0.3725 - val_ac

In [10]:
saved_model_path = '/content/drive/My Drive/saved_model'

# Save the model to the specified path

model.save(saved_model_path)

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define the image path
image_path = "/content/drive/My Drive/dog.png"

# Load and preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Assuming the model is trained for binary classification (AI-generated vs. real)
model = tf.keras.models.load_model('/content/drive/My Drive/saved_model')

# Preprocess the input image
image = load_and_preprocess_image(image_path)

# Make a prediction
prediction = model.predict(image)

# Assuming the output is a probability for AI-generated
is_ai_generated = prediction[0][0] > 0.5

if is_ai_generated:
    print("The image is AI-generated.")
else:
    print("The image is not AI-generated.")


1/1 [==============================] - 2s 2s/step
The image is AI-generated.


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define the image path
image_path = "/content/drive/My Drive/person.jpeg"

# Load and preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Assuming the model is trained for binary classification (AI-generated vs. real)
model = tf.keras.models.load_model('/content/drive/My Drive/saved_model')

# Preprocess the input image
image = load_and_preprocess_image(image_path)

# Make a prediction
prediction = model.predict(image)

# Assuming the output is a probability for AI-generated
is_ai_generated = prediction[0][0] > 0.5

if is_ai_generated:
    print("The image is AI-generated.")
else:
    print("The image is not AI-generated.")


1/1 [==============================] - 1s 976ms/step
The image is AI-generated.


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define the image path
image_path = "/content/drive/My Drive/srk.jpeg"

# Load and preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Assuming the model is trained for binary classification (AI-generated vs. real)
model = tf.keras.models.load_model('/content/drive/My Drive/saved_model')

# Preprocess the input image
image = load_and_preprocess_image(image_path)

# Make a prediction
prediction = model.predict(image)

# Assuming the output is a probability for AI-generated
is_ai_generated = prediction[0][0] > 0.5

if is_ai_generated:
    print("The image is AI-generated.")
else:
    print("The image is not AI-generated.")


1/1 [==============================] - 1s 937ms/step
The image is not AI-generated.


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define the image path
image_path = "/content/drive/My Drive/suhelsir.jpeg"

# Load and preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Assuming the model is trained for binary classification (AI-generated vs. real)
model = tf.keras.models.load_model('/content/drive/My Drive/saved_model')

# Preprocess the input image
image = load_and_preprocess_image(image_path)

# Make a prediction
prediction = model.predict(image)

# Assuming the output is a probability for AI-generated
is_ai_generated = prediction[0][0] > 0.5

if is_ai_generated:
    print("The image is AI-generated.")
else:
    print("The image is not AI-generated.")


1/1 [==============================] - 1s 950ms/step
The image is not AI-generated.


In [17]:
# Save the trained model
model.save('/content/drive/My Drive/resnet_model.keras')

In [19]:
# Save the trained model
model_save_path = '/content/drive/My Drive/resnet_model.keras'
model.save(model_save_path)

print(f'Model saved to {model_save_path}')



Model saved to /content/drive/My Drive/resnet_model.keras
